In [ ]:
peptide_filepath = "../Data/human/NEW_JMMdata_maxCVvalues.txt"

In [ ]:
import scipy
import numpy as np
import pandas as pd
from collections import defaultdict as dd
from matplotlib import pyplot as plt

In [ ]:
# Load data
# dat = pd.read_csv(peptide_filepath, index_col=0)
dat = pd.read_csv(peptide_filepath, low_memory=False, sep="\t") #read in data generated from R preprocessing
cols = dat.columns
new_cols = []
for c in cols:
    if c.isnumeric():
        new_cols.append("X" + str(c))
    elif c == "z_modseq":
        new_cols.append("SeqCharge")
    else:
        new_cols.append(c)
dat.columns = new_cols

xcols = [i for i in dat.columns if i.startswith("X")]
xcols_idx = [i for i, c in enumerate(dat.columns) if c.startswith("X")]

In [ ]:
# CV columns start with "X" due to some limitation in R; remove that and convert to ints
x = [int(i[1:]) for i in dat.columns if i.startswith("X")]

In [ ]:
x

In [ ]:
# Split peptides according to their CVmax; put full signal in dict
cvmax = dd(list)
for i in range(dat.shape[0]):
     pmax = x[np.argmax(dat.iloc[i,xcols_idx])]
     cvmax[pmax].append(dat.iloc[i,xcols_idx].values)

In [ ]:
def compute_meanvar_from_dist(value, freq) -> (float, float):
    """
    Computes the mean and variance of a population distribution (histogram).
    """
    m = np.sum(value*freq) / np.sum(freq)
    var = np.sum(freq * (value-m)**2) / np.sum(freq)
    return m, var

In [ ]:
def keep_connected_nonzero(arr):
    # Find the index of 1 in the array
    one_index = np.where(arr == 1)[0]
    if len(one_index) == 0:
        return np.zeros_like(arr)  # Return an array of zeros if 1 is not found
    one_index = one_index[0]
    
    # Traverse left and right to find continuous nonzero values
    left = one_index
    while left > 0 and arr[left - 1] != 0:
        left -= 1
    
    right = one_index
    while right < len(arr) - 1 and arr[right + 1] != 0:
        right += 1
    
    # Create a new array with zeros everywhere except the connected segment
    result = np.zeros_like(arr)
    result[left:right + 1] = arr[left:right + 1]
    return result

# Example array
example_array = np.array([0, 2, 3, 1, 4, 5, 0, 6])
result = keep_connected_nonzero(example_array)
print(result)  # Output: [0, 2, 3, 1, 4, 5, 0, 0]


In [ ]:
#counting on the times when ref is greater than dist
dist_list2 = dd(list)
pvals2 = dd(list)
refdist  = []
num_repetitions = 100
num_samples = 1000
randomly_keep_this_sig = []
randomly_keep_this_dist = []
randomly_keep_this_pval = []
random_k_value =82424
tracking_v = []

k = 0
for i in x:
     print(f"cv: {i}")
     for sig in cvmax[i]:
         sig = keep_connected_nonzero(sig) # keeps only the non-zero values that are continous with the value of 1 (CVmax)
         m, v = compute_meanvar_from_dist(x, sig)
         if np.isclose(v, 0, atol=1e-8): # setting this 
             continue
         tracking_v.append(v)
         #        g = scipy.stats.norm.pdf(x, i, np.sqrt(np.mean(all_vars)))  # more generous assumption: all distributions are the same and sufficiently summarized by CVmax
         g = scipy.stats.norm.pdf(x, m, np.sqrt(v))  # less generous assumption: distribution can be summarized by its mean and variance
         dist = scipy.stats.wasserstein_distance(sig/np.sum(sig), g/np.sum(g))
         p_count = 0
         if np.sum(g) == 0:
             continue
         for j in range(num_repetitions):
             ref = np.random.choice(x, size=num_samples, p=g/np.sum(g))
             ref_distr = np.histogram(ref, bins=x + [x[-1]*2-x[-2]])[0]
             ref_distance = scipy.stats.wasserstein_distance(ref_distr/np.sum(ref_distr), g/np.sum(g))
             if ref_distance > dist:
                 p_count += 1
             if k==random_k_value:
                 refdist.append(ref_distance)
                 randomly_keep_this_sig = sig
                 randomly_keep_this_dist = dist
         pvals2[i].append(p_count/(j+1))
         if k==random_k_value:
             randomly_keep_this_pval = p_count/(j+1)
         dist_list2[i].append(dist)
         k+=1 # counter to keep a random distribution from the middle

In [ ]:
tmp = plt.hist(tracking_v, bins=10000)
plt.xlim(-2,40)

In [ ]:
plt.plot(randomly_keep_this_sig)

In [ ]:
plt.hist(refdist)
plt.axvline(x=randomly_keep_this_dist, color='r')
plt.title('random subsampled distances versus the observed distance')

In [ ]:
from statsmodels.stats.multitest import multipletests

def apply_bh_correction(pvalue_dict):
    """Applies Benjamini-Hochberg correction to all p-values in the dictionary.

    Args:
        pvalue_dict: A dictionary where keys map to lists of p-values.

    Returns:
        A new dictionary with the same structure, but with BH-corrected p-values.
    """

    # Flatten all p-values into a single list
    all_pvalues = []
    for pvalue_list in pvalue_dict.values():
        all_pvalues.extend(pvalue_list)

    # Apply BH correction
    _, bh_corrected_pvalues, _ , _ = multipletests(all_pvalues, alpha=0.05, method='fdr_bh')

    # Reassign corrected p-values to the original dictionary structure
    corrected_pvalue_dict = {}
    i = 0
    for key, pvalue_list in pvalue_dict.items():
        corrected_pvalue_dict[key] = bh_corrected_pvalues[i:i+len(pvalue_list)]
        i += len(pvalue_list)

    return corrected_pvalue_dict

# Example usage:
pvalue_dict = {
    'key1': [0.01, 0.02, 0.03],
    'key2': [0.04, 0.05, 0.06],
    # ... other keys and p-value lists
}

corrected_pvalue_dict = apply_bh_correction(pvalue_dict)
print(corrected_pvalue_dict)

In [ ]:
corrected_pvals = apply_bh_correction(pvals2)

In [ ]:
for i in x:
    plt.bar(-i, np.sum(np.array(pvals2[i])<=0.01) / len(pvals2[i]), color="#3b528b", width=5, edgecolor="black")
plt.xlabel("CV", fontsize=20)
plt.ylabel("Fraction", fontsize=20)
plt.ylim(0,1)
plt.title("Fraction of Non-Normal Peptides, Uncorrected", fontsize=22)
plt.savefig("../plots/fraction_nonnormal_human_pt01.png", bbox_inches="tight")
plt.savefig("../plots/fraction_nonnormal_human_pt01.svg", bbox_inches="tight")

In [ ]:
for i in x:
    plt.bar(-i, np.sum(np.array(corrected_pvals[i])<=0.01) / len(corrected_pvals[i]), color="#3b528b", width=5, edgecolor="black")
plt.xlabel("CV", fontsize=20)
plt.ylabel("Fraction", fontsize=20)
plt.ylim(0,1)
plt.title("Fraction of Non-Normal Peptides, Corrected", fontsize=20)
plt.savefig("../plots/fraction_nonnormal_human_pt01_corrected.png", bbox_inches="tight")
plt.savefig("../plots/fraction_nonnormal_human_pt01_corrected.svg", bbox_inches="tight")